# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))    
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = 'data/song_data'
#get total number of files found
num_files = len(song_files)
print('{} files found {}'.format(song_files, filepath))

['/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json', '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json', '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json', '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json', '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json', '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json', '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json', '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json', '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json', '/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json', '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json', '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAVO128F93133D4-checkpoint.json', '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAFD128F92F423A-checkpoint.json', '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpoint.json', '/home/worksp

In [6]:
df = pd.read_json('/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json',lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR558FS1187FB45658,NaN,,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id','title','artist_id','year','duration']].iloc[0].values.tolist()
song_data
type(song_data[4])

numpy.float64

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
print(song_table_insert)

INSERT INTO songs (song_id,title,artist_id,year,duration) VALUES (%s,%s,%s,%s,%s)


In [9]:
cur.execute(song_table_insert, ([song_data[0],song_data[1],song_data[2],song_data[3].item(),song_data[4].item()]))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].iloc[0].values.tolist()
artist_data
type(artist_data[3])

numpy.float64

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, ([artist_data[0],artist_data[1],artist_data[2],artist_data[3].item(),artist_data[4].item()]))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [4]:
log_files = get_files('data/log_data')

In [5]:
filepath = 'data/log_data'
num_files = len(log_files)
print('{} files found {}'.format(log_files, filepath))

['/home/workspace/data/log_data/2018/11/2018-11-29-events.json', '/home/workspace/data/log_data/2018/11/2018-11-03-events.json', '/home/workspace/data/log_data/2018/11/2018-11-01-events.json', '/home/workspace/data/log_data/2018/11/2018-11-05-events.json', '/home/workspace/data/log_data/2018/11/2018-11-02-events.json', '/home/workspace/data/log_data/2018/11/2018-11-24-events.json', '/home/workspace/data/log_data/2018/11/2018-11-13-events.json', '/home/workspace/data/log_data/2018/11/2018-11-26-events.json', '/home/workspace/data/log_data/2018/11/2018-11-06-events.json', '/home/workspace/data/log_data/2018/11/2018-11-14-events.json', '/home/workspace/data/log_data/2018/11/2018-11-23-events.json', '/home/workspace/data/log_data/2018/11/2018-11-09-events.json', '/home/workspace/data/log_data/2018/11/2018-11-10-events.json', '/home/workspace/data/log_data/2018/11/2018-11-17-events.json', '/home/workspace/data/log_data/2018/11/2018-11-20-events.json', '/home/workspace/data/log_data/2018/11/

In [6]:
df = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-02-events.json', lines=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,1541127957796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48
6,None,Logged In,Aiden,M,0,Hess,NaN,free,"La Crosse-Onalaska, WI-MN",GET,Home,1540829025796,170,None,200,1541129561796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86
7,Dalto,Logged In,Aiden,M,1,Hess,190.40608,free,"La Crosse-Onalaska, WI-MN",PUT,NextSong,1540829025796,170,Falta Te Dizer,200,1541129674796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86
8,Kanye West,Logged In,Makinley,F,0,Jones,278.88281,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1541091973796,118,Family Business,200,1541135741796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",17
9,Jason Mraz & Colbie Caillat,Logged In,Kevin,M,0,Arellano,189.62240,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,187,Lucky (Album Version),200,1541137949796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [7]:
df = df.query('page=="NextSong"')
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,1541127957796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48
7,Dalto,Logged In,Aiden,M,1,Hess,190.40608,free,"La Crosse-Onalaska, WI-MN",PUT,NextSong,1540829025796,170,Falta Te Dizer,200,1541129674796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86
8,Kanye West,Logged In,Makinley,F,0,Jones,278.88281,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1541091973796,118,Family Business,200,1541135741796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",17
9,Jason Mraz & Colbie Caillat,Logged In,Kevin,M,0,Arellano,189.62240,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,187,Lucky (Album Version),200,1541137949796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
10,Liz Callaway,Logged In,Lily,F,0,Koch,175.43791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1541048010796,172,Journey To The Past (LP Version),200,1541149281796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
11,The Decemberists,Logged In,Lily,F,1,Koch,242.59873,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1541048010796,172,Everything I Try to Do_ Nothing Seems to Turn ...,200,1541149456796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [8]:
t = pd.to_datetime(df['ts'], unit ='ms')
t.dt.time.head()

0    01:25:34.796000
2    01:30:41.796000
3    01:34:17.796000
4    02:42:48.796000
5    03:05:57.796000
Name: ts, dtype: object

In [9]:
time_data = (t.dt.time,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday)
column_labels = ('timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday')
type(column_labels)
timeData = dict(zip(column_labels,time_data))
print(timeData)

{'timestamp': 0      01:25:34.796000
2      01:30:41.796000
3      01:34:17.796000
4      02:42:48.796000
5      03:05:57.796000
7      03:34:34.796000
8      05:15:41.796000
9      05:52:29.796000
10     09:01:21.796000
11     09:04:16.796000
13     09:08:18.796000
14     09:12:32.796000
15     09:13:37.796000
16     09:16:16.796000
17     09:19:15.796000
18     09:22:43.796000
19     09:26:49.796000
20     09:31:00.796000
21     09:35:25.796000
22     09:40:11.796000
23     09:42:23.796000
24     09:47:01.796000
25     09:50:23.796000
26     09:54:59.796000
27     09:57:56.796000
28     10:02:20.796000
29     10:04:05.796000
30     10:07:43.796000
31     10:11:24.796000
32     10:13:59.796000
            ...       
139    16:22:13.796000
140    16:24:54.796000
141    16:27:56.796000
142    16:30:54.796000
143    16:32:07.796000
144    16:32:51.796000
145    16:34:47.796000
146    16:35:00.796000
147    16:38:55.796000
148    16:38:59.796000
149    16:46:02.796000
150    16:49:44.7960

In [31]:
time_df = pd.DataFrame.from_dict(timeData)
time_df.head()

,timestamp,hour,day,week of year,month,year,weekday
2,21:01:46.796000,21,1,44,11,2018,3
4,21:05:52.796000,21,1,44,11,2018,3
5,21:08:16.796000,21,1,44,11,2018,3
6,21:11:13.796000,21,1,44,11,2018,3
7,21:17:33.796000,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
''''for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()'''

from io import StringIO

def copy_to_postgres(conn, time_df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    time_df.to_csv(buffer, index_label='Id', header=False)
    buffer.seek(0)
    
    print(time_df)
    
    cursor = conn.cursor()
    try:
        cur.copy_from(buffer, table, sep = ",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()


# Run the execute_many strategy
copy_from_stringio(conn, time_df, 'time')



          timestamp  hour  day  week of year  month  year  weekday
2   21:01:46.796000    21    1            44     11  2018        3
4   21:05:52.796000    21    1            44     11  2018        3
5   21:08:16.796000    21    1            44     11  2018        3
6   21:11:13.796000    21    1            44     11  2018        3
7   21:17:33.796000    21    1            44     11  2018        3
8   21:24:53.796000    21    1            44     11  2018        3
9   21:28:54.796000    21    1            44     11  2018        3
10  21:42:00.796000    21    1            44     11  2018        3
12  21:52:05.796000    21    1            44     11  2018        3
13  21:55:25.796000    21    1            44     11  2018        3
14  22:23:14.796000    22    1            44     11  2018        3
copy_from_stringio() done


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
13,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [10]:
songplay_columns = ('start_time','user_id','level','song_id','artist_id','session_id','location','user_agent')
songplay_data =[]
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    #print(row.song)
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    #start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
    songplay_dic =dict(zip(songplay_columns,(row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)))
    songplay_data.append(songplay_dic)
                       
    #songplay_data.append([row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent])
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()
print(songplay_data)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[{'start_time': 1541121934796, 'user_id': 101, 'level': 'free', 'song_id': None, 'artist_id': None, 'session_id': 184, 'location': 'New Orleans-Metairie, LA', 'user_agent': '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/53

In [45]:
print (type(songplay_data))
songplay_df = pd.DataFrame(songplay_data)
list(songplay_df.columns.values)
songplay_df = songplay_df.reindex(columns=songplay_columns)
songplay_df

<class 'list'>


,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,1541121934796,101,free,None,None,184,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
1,1541122241796,83,free,None,None,82,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,1541122457796,83,free,None,None,82,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,1541126568796,66,free,None,None,153,"Harrisburg-Carlisle, PA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,1541127957796,48,free,None,None,47,"Salinas, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
5,1541129674796,86,free,None,None,170,"La Crosse-Onalaska, WI-MN","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6,1541135741796,17,free,None,None,118,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
7,1541137949796,66,free,None,None,187,"Harrisburg-Carlisle, PA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
8,1541149281796,15,paid,None,None,172,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
9,1541149456796,15,paid,None,None,172,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.